<a href="https://colab.research.google.com/github/amarsinghen/question_answering_kaggle/blob/master/tensorflow2_0_question_answering_competition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

In [8]:
import tensorflow as tf
tf.__version__

'2.1.0-rc1'

In [2]:
# Run this cell and select the kaggle.json file downloaded
# from the Kaggle account settings page.
from google.colab import files
uploaded = files.upload()

MessageError: ignored

In [0]:
!pip install -q kaggle

In [0]:
# The Kaggle API client expects this file to be in ~/.kaggle,
# so move it there.
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [0]:
# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json

In [12]:
!kaggle competitions download -c tensorflow2-question-answering

100% 4.78M/4.78M [00:00<00:00, 39.0MB/s]

100% 4.45G/4.46G [01:02<00:00, 91.6MB/s]
100% 4.46G/4.46G [01:02<00:00, 76.5MB/s]
  0% 0.00/18.2k [00:00<?, ?B/s]
100% 18.2k/18.2k [00:00<00:00, 18.7MB/s]


In [0]:
import zipfile
local_zip = 'simplified-nq-train.jsonl.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

In [0]:
import pandas as pd
nq_train_df = pd.read_json('/tmp/simplified-nq-train.jsonl', lines=True)

In [1]:
!ls -l /tmp/simplified-nq-train.jsonl

-rw-r--r-- 1 root root 17454394481 Jan 12 19:02 /tmp/simplified-nq-train.jsonl


In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

In [0]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [0]:
df = spark.read.json('/tmp/simplified-nq-train.jsonl')

In [20]:
df.show()

+--------------------+--------------------+--------------------+--------------------+----------------------+--------------------+
|         annotations|       document_text|        document_url|          example_id|long_answer_candidates|       question_text|
+--------------------+--------------------+--------------------+--------------------+----------------------+--------------------+
|[[593165450220027...|Email marketing -...|https://en.wikipe...| 5655493461695504401|  [[170, 14, true],...|which is the most...|
|[[120348741537837...|The Mother ( How ...|https://en.wikipe...| 5328212470870865242|  [[212, 28, true],...|how i.met your mo...|
|[[105271230098927...|Human fertilizati...|https://en.wikipe...| 4435104480114867852|  [[225, 14, true],...|what type of fert...|
|[[146347963651525...|List of National ...|https://en.wikipe...| 5289242154789678439|  [[469, 28, true],...|who had the most ...|
|[[110385499948886...|Roanoke Colony - ...|https://en.wikipe...| 5489863933082811018|  [[8

In [24]:
df.head(1)

[Row(annotations=[Row(annotation_id=Decimal('593165450220027640'), long_answer=Row(candidate_index=54, end_token=2019, start_token=1952), short_answers=[Row(end_token=1969, start_token=1960)], yes_no_answer='NONE')], document_text="Email marketing - Wikipedia <H1> Email marketing </H1> Jump to : navigation , search <Table> <Tr> <Td> </Td> <Td> ( hide ) This article has multiple issues . Please help improve it or discuss these issues on the talk page . ( Learn how and when to remove these template messages ) <Table> <Tr> <Td> </Td> <Td> This article needs additional citations for verification . Please help improve this article by adding citations to reliable sources . Unsourced material may be challenged and removed . ( September 2014 ) ( Learn how and when to remove this template message ) </Td> </Tr> </Table> <Table> <Tr> <Td> </Td> <Td> This article possibly contains original research . Please improve it by verifying the claims made and adding inline citations . Statements consisting